In [44]:
import os
import cv2
import shutil
import PIL
import random
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [57]:
BASE_DIR = '../'
MODEL_DIR = BASE_DIR+'model/'
SOURCE_DIR = BASE_DIR + 'src_data/'
NO_MASK_DIRS_2 = [SOURCE_DIR + 'RMFD_part_2/RWMFD_part_2_pro/', SOURCE_DIR + 'RMFD_part_2-1/RWMFD_part_2_pro/',
                  SOURCE_DIR + 'RMFD_part_2-2/RWMFD_part_2_pro/', SOURCE_DIR + 'RMFD_part_2-3/RWMFD_part_2_pro/']
NO_MASK_DIRS_3 =[SOURCE_DIR + 'RMFD_part_3/single2-0/', SOURCE_DIR + 'RMFD_part_3-1/single2-0/',
                SOURCE_DIR + 'RMFD_part_3-2/single2-0/']
DATASET_DIR = BASE_DIR + 'dataset/'

DATA_DIR = Path(DATASET_DIR)
IMG_HEIGHT = 256
IMG_WIDTH = 256

In [4]:
def get_face(img, x, y, w, h) -> Image.Image: 
    crop_img = img[y:y+h, x:x+w]
    rgb_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_img)
    return pil_img

In [15]:
Path(DATASET_DIR).mkdir(parents=True, exist_ok=True)
Path(DATASET_DIR+'correctly-masked-real').mkdir(parents=True, exist_ok=True)
Path(DATASET_DIR+'correctly-masked-crop').mkdir(parents=True, exist_ok=True)

In [26]:
try:
    os.remove(NO_MASK_DIR_2 + '.DS_Store')
except:
    pass
for no_mask_dir in NO_MASK_DIRS_2:
    for subdir in tqdm(os.listdir(no_mask_dir)):
        full_subdir = no_mask_dir + subdir + '/'
        new_name_prefix = full_subdir + subdir
        [shutil.copy(full_subdir + image, DATASET_DIR+'correctly-masked-real/'+image) for image in os.listdir(full_subdir)]

100%|██████████| 394/394 [00:01<00:00, 282.80it/s]


In [41]:
# crop images
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for image in tqdm(os.listdir(DATASET_DIR+'correctly-masked-real')):
    img = cv2.imread(DATASET_DIR+'correctly-masked-real/'+image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        faces = img[y:y + h, x:x + w]
        image_name = DATASET_DIR+'correctly-masked-crop/'+image+'-face.jpeg'
        cv2.imwrite(image_name, faces)

 60%|██████    | 2276/3773 [00:29<00:19, 76.27it/s]


error: OpenCV(4.5.2) /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-2fjztvm5/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [48]:
def convert_images_to_jpeg(s_dir, img_width, img_height):
    s_list= os.listdir(s_dir)
    for clazz in s_list:
        clazz_path=os.path.join (s_dir, clazz)
        print('[INFO] - processing class directory ', clazz)
        if os.path.isdir(clazz_path):
            file_list=os.listdir(clazz_path)
            for f in file_list:               
                f_path=os.path.join(clazz_path,f)
                index=f_path.rfind('.')
                f_path_no_ext=f_path[:index].lower()
                img = Image.open(f_path)
                img = img.convert('RGB')
                newsize = (img_width, img_height)
                img = img.resize(newsize)
                img.save(f_path_no_ext+'.jpeg')

def remove_jpg(s_dir):
    s_list= os.listdir(s_dir)
    for clazz in s_list:
        clazz_path=os.path.join(s_dir, clazz)
        print('[INFO] - All .jpg files removed in', clazz_path)
        [os.remove(clazz_path+'/'+image) for image in os.listdir(clazz_path) if (clazz_path+'/'+image).endswith('.jpg')]

source_dir =r"{}".format(DATASET_DIR)
os.remove(DATASET_DIR + '.DS_Store')
convert_images_to_jpeg(source_dir, IMG_WIDTH, IMG_HEIGHT)
os.remove(DATASET_DIR + '.DS_Store')
remove_jpg(source_dir)

[INFO] - processing class directory  correctly-masked
[INFO] - processing class directory  not-masked-crop
[INFO] - processing class directory  not-masked
[INFO] - processing class directory  incorrectly-masked
[INFO] - processing class directory  not-masked-real


UnidentifiedImageError: cannot identify image file '../dataset/not-masked-real/0041500415011.jpg'

In [59]:
ds = tf.keras.preprocessing.image_dataset_from_directory(
    Path(BASE_DIR+'tmp/'),
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    seed = 123)

ds_size = sum(1 for _ in ds.unbatch().take(-1))
print(f'[DEBUG] - ds_size: {ds_size}')
class_names = ds.class_names
ds = ds.shuffle(10000, seed=12)
print(f'[INFO] - Class names: {class_names}')

Found 8344 files belonging to 3 classes.
[DEBUG] - ds_size: 8344
[INFO] - Class names: ['correctly-masked', 'incorrectly-masked', 'not-masked']


In [60]:
x = []
y = []
for image, label in tqdm(ds.unbatch().take(-1)):
    img = image.numpy().astype("uint8")
    x.append(img)
    label = label.numpy()
    y.append(label)
    
x = np.array(x)
y = np.array(y)

8344it [00:17, 467.32it/s] 


In [62]:
# NB: rinominare cartella per avere la label corretta
model = keras.models.load_model(MODEL_DIR)
predictions = np.argmax(model.predict(x), axis=-1)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y, predictions, 
                            target_names = ['correctly-masked (Class 0)','not-masked (Class 1)', 'uncorrectly-masked (Class 2)']))

                              precision    recall  f1-score   support

  correctly-masked (Class 0)       0.90      0.18      0.30       532
        not-masked (Class 1)       0.00      0.00      0.00      5771
uncorrectly-masked (Class 2)       0.00      0.00      0.00      2041

                    accuracy                           0.01      8344
                   macro avg       0.30      0.06      0.10      8344
                weighted avg       0.06      0.01      0.02      8344

